In [59]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import Plotting Libararies
import seaborn as sns
import matplotlib.pyplot as plt

# Model Evaluation Libraries
from sklearn.metrics import classification_report, confusion_matrix

import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

In [61]:
dataset = pd.read_csv("sarcasm_dataset.csv")
dataset.head()

,Unnamed: 0,tweet,sarcastic,sarcasm,irony,satire,understatement,overstatement,rhetorical_question
0,0,The only thing I got from college is a caffeine addiction,1,0.0,1.0,0.0,0.0,0.0,0.0
1,1,I love it when professors draw a big question mark next to my answer on an exam because I’m always like yeah I don’t either ¯\_(ツ)_/¯,1,1.0,0.0,0.0,0.0,0.0,0.0
2,2,Remember the hundred emails from companies when Covid started getting real? I’ve gotten three in regards to support for protests. And only @SavageXFenty shared helpful links and actually said black lives matter... we love capitalism 🥰🙌🏼,1,0.0,1.0,0.0,0.0,0.0,0.0
3,3,Today my pop-pop told me I was not “forced” to go to college 🙃 okay sure sureeee,1,1.0,0.0,0.0,0.0,0.0,0.0
4,4,"@VolphanCarol @littlewhitty @mysticalmanatee I did too, and I also reported Cancun Cruz not worrying about the heartbeats of his constituents without electricity or heat when he fled to Mexico.",1,1.0,0.0,0.0,0.0,0.0,0.0


In [39]:
#preprocessing
data = dataset[['tweet', 'sarcastic']]


In [40]:
#expanding the dispay of text sms column
pd.set_option('display.max_colwidth', -1)
#using only v1 and v2 column

data.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,tweet,sarcastic
0,The only thing I got from college is a caffeine addiction,1
1,I love it when professors draw a big question mark next to my answer on an exam because I’m always like yeah I don’t either ¯\_(ツ)_/¯,1
2,Remember the hundred emails from companies when Covid started getting real? I’ve gotten three in regards to support for protests. And only @SavageXFenty shared helpful links and actually said black lives matter... we love capitalism 🥰🙌🏼,1
3,Today my pop-pop told me I was not “forced” to go to college 🙃 okay sure sureeee,1
4,"@VolphanCarol @littlewhitty @mysticalmanatee I did too, and I also reported Cancun Cruz not worrying about the heartbeats of his constituents without electricity or heat when he fled to Mexico.",1


In [41]:
df = data.copy()

In [42]:
df["text_lower"] = df["tweet"].str.lower()
df.head()

,tweet,sarcastic,text_lower
0,The only thing I got from college is a caffeine addiction,1,the only thing i got from college is a caffeine addiction
1,I love it when professors draw a big question mark next to my answer on an exam because I’m always like yeah I don’t either ¯\_(ツ)_/¯,1,i love it when professors draw a big question mark next to my answer on an exam because i’m always like yeah i don’t either ¯\_(ツ)_/¯
2,Remember the hundred emails from companies when Covid started getting real? I’ve gotten three in regards to support for protests. And only @SavageXFenty shared helpful links and actually said black lives matter... we love capitalism 🥰🙌🏼,1,remember the hundred emails from companies when covid started getting real? i’ve gotten three in regards to support for protests. and only @savagexfenty shared helpful links and actually said black lives matter... we love capitalism 🥰🙌🏼
3,Today my pop-pop told me I was not “forced” to go to college 🙃 okay sure sureeee,1,today my pop-pop told me i was not “forced” to go to college 🙃 okay sure sureeee
4,"@VolphanCarol @littlewhitty @mysticalmanatee I did too, and I also reported Cancun Cruz not worrying about the heartbeats of his constituents without electricity or heat when he fled to Mexico.",1,"@volphancarol @littlewhitty @mysticalmanatee i did too, and i also reported cancun cruz not worrying about the heartbeats of his constituents without electricity or heat when he fled to mexico."


In [43]:
tokenizer = nltk.RegexpTokenizer(r"\w+")
df["tokenized"]  = tokenizer.tokenize(df["text_lower"])

TypeError: expected string or bytes-like object

In [46]:
from sklearn.model_selection import train_test_split
# create dataset
X = df['text_lower']
Y = df["sarcastic"]
print(X.shape)
print(Y.shape)

# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3468,)
(3468,)
(2323,) (1145,) (2323,) (1145,)


In [47]:
import numpy as np

with open("glove.6B.50d.txt", "rb") as lines:
    w2v = {line.split()[0]: np.array(map(float, line.split()[1:]))
           for line in lines}

In [36]:
import gensim
# let X be a list of tokenized texts (i.e. list of lists of tokens)
model = gensim.models.Word2Vec(X, size=100)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

NameError: name 'X' is not defined

In [53]:
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.values().next())

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [54]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.values().next())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier

etree_w2v = Pipeline([
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_w2v_tfidf = Pipeline([
    ("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)),
    ("extra trees", ExtraTreesClassifier(n_estimators=200))])

AttributeError: 'dict_values' object has no attribute 'next'